In [1]:
from __future__ import division
from __future__ import print_function
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, roc_curve, auc
from sklearn import cross_validation
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as plt
%matplotlib inline

import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import *

import re
from HTMLParser import HTMLParser
import datetime
import cPickle as pickle
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# Files
all_file = '/Users/lekha/galvanize/capstone/projectRiley/data/withindgroup/allwithindgroup.txt'
tech_file = '/Users/lekha/galvanize/capstone/projectRiley/data/cleandatagender1000.txt'


In [3]:
df = pd.read_csv(all_file, sep="|")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31415 entries, 0 to 31414
Data columns (total 18 columns):
first_name          31415 non-null object
full_name           31415 non-null object
html                31415 non-null object
industry            31415 non-null object
location            31415 non-null object
summary             31414 non-null object
title               31415 non-null object
counter             31415 non-null float64
source              31415 non-null object
full_name_fields    31415 non-null object
name_fields         31415 non-null int64
gender              31415 non-null object
gender_type         31415 non-null object
gender_forced       31415 non-null object
name_counts         31415 non-null int64
loc_fields          31415 non-null object
state               31415 non-null object
ind_group           31415 non-null int64
dtypes: float64(1), int64(3), object(14)
memory usage: 4.6+ MB


In [13]:
def get_ind_group(ind):
    ind = ind.strip().lower()
    ind_groups = { 'missing': 0,
                   'higher education': 1,
                   'research': 1,
                   'computer software': 2,
                   'information technology and services': 2,
                   'internet': 2,
                   'design': 2,
                   'mechanical or industrial engineering': 2,
                   'aviation aerospace': 2,
                   'electrical electronic manufacturing': 2,
                   'civil engineering': 2,
                   'telecommunications': 2,
                   'graphic design': 2,
                   'information services': 2,
                   'semiconductors': 2,
                   'computer hardware': 2,
                   'logistics and supply chain': 2,
                   'automotive': 2,
                   'program development': 2,
                   'computer games': 2,
                   'computer network security': 2,
                   'nanotechnology': 2,
                   'animation': 2,
                   'nanotechnologies': 2 }

    result = ind_groups.get(ind, 3)
    return result


def strip_punc(x):
    x = x.lower()
    pattern = re.compile('([^\s\w]|_)+')
    x = pattern.sub(' ', x)
    return x


def get_industry(df):
    df['industry'] = df['industry'].apply(lambda x: strip_punc(x))
    industries = df.industry.value_counts()
    inds = industries.index.tolist()
    df['ind_group'] = df['industry'].apply(lambda x:get_ind_group(x))
    return df


In [14]:
df['ind_group'] = df['industry'].apply(lambda x:get_ind_group(x))

In [20]:
df.ind_group.value_counts()

3    20277
2     5634
1     4320
0     1184
Name: ind_group, dtype: int64

In [16]:
industries = df.industry.value_counts()
inds = industries.index.tolist()
IndN = min(len(inds), 50)

In [18]:
IndN

50